In [12]:
from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)

from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI


dog_schema = {
    "type": "function",
    "function": {
        "name": "record_dog",
        "description": "Record some identifying information about a dog.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "description": "The dog's name",
                    "type": "string"
                },
                "color": {
                    "description": "The dog's color",
                    "type": "string"
                },
                "fav_food": {
                    "description": "The dog's favorite food",
                    "type": "string"
                }
            },
            "required": ["name", "color"]
        }
    }
}


structured_llm = create_structured_output_runnable(
    dog_schema,
    llm,
    mode="openai-tools",
    enforce_function_usage=True,
)
res = structured_llm.invoke("Harry was a chubby brown beagle who loved chicken")
print(res)
# -> {'name': 'Harry', 'color': 'brown', 'fav_food': 'chicken'}
# -> RecordDog(name="Harry", color="brown", fav_food="chicken")

None


In [3]:
import json

data = {
    "created_at": "2024-06-13T12:00:00Z",  # Example timestamp
    "event": "AgentWantsUpdatedStateEvent",
    "details": {"sender_id": None}  # Assuming `sender_id` should have a value
}

print(json.dumps(data))


{"created_at": "2024-06-13T12:00:00Z", "event": "AgentWantsUpdatedStateEvent", "details": {"sender_id": null}}


In [13]:
from typing import Optional

from langchain.chains import create_structured_output_runnable
from langchain.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)


from kor import create_extraction_chain, Object, Text 

schema = Object(
    id="player",
    description=(
        "User is controlling a music player to select songs, pause or start them or play"
        " music by a particular artist."
    ),
    attributes=[
        Text(
            id="song",
            description="User wants to play this song",
            examples=[],
            many=True,
        ),
        Text(
            id="album",
            description="User wants to play this album",
            examples=[],
            many=True,
        ),
        Text(
            id="artist",
            description="Music by the given artist",
            examples=[("Songs by paul simon", "paul simon")],
            many=True,
        ),
        Text(
            id="action",
            description="Action to take one of: `play`, `stop`, `next`, `previous`.",
            examples=[
                ("Please stop the music", "stop"),
                ("play something", "play"),
                ("play a song", "play"),
                ("next song", "next"),
            ],
        ),
    ],
    many=False,
)

chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')
res = chain.invoke("play songs by paul simon and led zeppelin and the doors")
player_data = res['text']['data']['player']
print(player_data)


/root/.cache/pypoetry/virtualenvs/genworlds-KfguBBuL-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


{'text': {'data': {}, 'raw': '<json={"player": {"song": [], "artist": ["paul simon", "led zeppelin", "the doors"]}}}</json>', 'errors': [], 'validated_data': {}}}


In [11]:
import enum
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Number
import pydantic
from typing import List
from kor import from_pydantic
from pydantic import BaseModel, Field
from typing import Optional


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)


class Action(enum.Enum):
    play = "play"
    stop = "stop"
    previous = "previous"
    next_ = "next"


class MusicRequest(BaseModel):
    song: Optional[List[str]] = Field(
        default=None, description="The song(s) that the user would like to be played."
    )
    album: Optional[List[str]] = Field(
        default=None, description="The album(s) that the user would like to be played."
    )
    artist: Optional[List[str]] = Field(
        default=None,
        description="The artist(s) whose music the user would like to hear.",
        examples=[("Songs by paul simon", "paul simon")],
    )
    action: Optional[Action] = Field(
        default=None,
        description="The action that should be taken; one of `play`, `stop`, `next`, `previous`",
        examples=[
            ("Please stop the music", "stop"),
            ("play something", "play"),
            ("play a song", "play"),
            ("next song", "next"),
        ],
    )

schema, validator = from_pydantic(MusicRequest)
chain = create_extraction_chain(
    llm, schema, encoder_or_encoder_class="json", validator=validator
)
print(chain)
#print(chain.prompt.format_prompt(text="[user input]").to_string())
chain.invoke("stop the music now")

prompt=ExtractionPromptTemplate(input_variables=['text'], output_parser=KorParser(encoder=<kor.encoders.json_data.JSONEncoder object at 0x7f39bf41ae90>, schema_=Object(id='musicrequest', description='', many=False, attributes=[Text(id='song', description='The song(s) that the user would like to be played.', many=True, examples=()), Text(id='album', description='The album(s) that the user would like to be played.', many=True, examples=()), Text(id='artist', description='The artist(s) whose music the user would like to hear.', many=True, examples=[('Songs by paul simon', 'paul simon')]), Selection(id='action', description='The action that should be taken; one of `play`, `stop`, `next`, `previous`', many=False, options=[Option(id='play', description='', many=False, examples=()), Option(id='stop', description='', many=False, examples=()), Option(id='previous', description='', many=False, examples=()), Option(id='next', description='', many=False, examples=())], examples=[('Please stop the 

{'text': {'data': {'musicrequest': {'action': 'stop'}},
  'raw': '<json>{"musicrequest": {"action": "stop"}}</json>',
  'errors': [],
  'validated_data': MusicRequest(song=None, album=None, artist=None, action=<Action.stop: 'stop'>)}}

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("http://www.moe.gov.cn/jyb_sjzl/moe_560/2022/quanguo/202401/t20240110_1099483.html",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()
len(docs[0].page_content)
print(docs[0].page_content[:500])

SyntaxError: incomplete input (249806899.py, line 50)

In [37]:
import enum

from langchain_openai import ChatOpenAI
from kor import create_extraction_chain, Object, Text, Number
import pydantic
from typing import List
from kor import from_pydantic
from pydantic import BaseModel, ConfigDict, Field
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.structured_output import create_structured_output_runnable


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)

class PlanNextAction(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    """Plans for the next action to be executed by the agent."""

    action_name: str = Field(
        ...,
        description="Selects the action name of the next action to be executed from the list of available action names.",
    )
    is_action_valid: bool = Field(
        ..., 
        description="Determines whether the next action is valid or not."
    )
    is_action_valid_reason: str = Field(
        ...,
        description="Then explains the rationale of whether it is valid or not valid action.",
    )
    new_plan: List[str] = Field(
        ..., 
        description="The new plan to execute to achieve the goals."
    )
    # class Config:
    #     arbitrary_types_allowed = True


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are {agent_name}, {agent_description}.\n"),
        (
            "system",
            "You are embedded in a simulated world with those properties {agent_world_state}\n",
        ),
        ("system", "Those are your goals: \n{goals}\n"),
        (
            "system",
            "And this is the previous plan to achieve the goals: \n{plan}\n",
        ),
        (
            "system",
            "Here is your memories of all the events that you remember from being in this simulation: \n{memory}\n",
        ),
        (
            "system",
            "Those are the available actions that you can choose from: \n{available_actions}\n",
        ),
        ("human", "{footer}\n"),
    ]
)

input_context = {
    "agent_name": "self.agent_state.name",
    "agent_description": "self.agent_state.description",
    "agent_world_state": "self.agent_state.host_world_prompt",
    "goals": "self.agent_state.goals",
    "plan": "self.agent_state.plan",
    "memory": "self.agent_state.last_retrieved_memory",
    "available_actions": "action_schemas_full_string",
    "footer": """Select the next action which must be a value of the available actions that you can choose from based on previous context.
    Also select whether the action is valid or not, and if not, why.
    And finally, state a new updated plan that you want to execute to achieve your goals. If your next action is going to sleep, then you don't need to state a new plan.
            """
}
# schema, validator = from_pydantic(PlanNextAction)
# chains = create_extraction_chain(
#     llm, schema, encoder_or_encoder_class="json", validator=validator
# )
# print(chains)
chain = create_structured_output_runnable(
    PlanNextAction,
    llm,
    prompt,
    mode="openai-json",
    enforce_function_usage=False,
    return_single=True
)

#chain = prompt | self.llm.with_structured_output(schema=PlanNextAction)
response = chain.invoke(input_context)

print("response1 ", response)

response1  {'action': 'ask_question', 'valid': True}


In [2]:
#从网址提取信息
import re
from bs4 import BeautifulSoup, SoupStrainer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

# HTML elements to keep in the parsing
bs4_strainer = SoupStrainer(["table"])

# Initialize WebBaseLoader with the specified URL and BeautifulSoup options
loader = WebBaseLoader(
    web_paths=("http://www.moe.gov.cn/jyb_sjzl/moe_560/2022/quanguo/202401/t20240110_1099483.html",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Load the document
docs = loader.load()

# 提取文本内容
page_content = docs[0].page_content  # 直接访问 Document 对象的 page_content 属性

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(page_content, "html.parser")

# 提取所有的文本内容
text_content = soup.get_text()

# 清理不需要的空白字符和换行符
cleaned_text = " ".join(text_content.split())

# 打印清理后的文本内容
print(cleaned_text)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)
all_splits[1].metadata

# 使用正则表达式提取数据
pattern = r'\d+\.\s*([\u4e00-\u9fa5\s（）a-zA-Z0-9]+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)\s+(\d+)'
matches = re.findall(pattern, cleaned_text)

# 将匹配结果转换为表格格式输出
print(f"{'分类':<35} {'总计':<8} {'校外教师':<8} {'行业导师':<8} {'外籍教师':<8} {'离退休人员':<8} {'附属中小学幼儿园教职工':<8} {'专任教师':<8} {'行政人员':<8} {'教辅人员':<8} {'工勤人员':<8} {'专职科研人员':<8} {'其他附设机构人员':<8}")
print("=" * 125)
for match in matches:
    category = match[0].strip()
    total = match[1]
    part_time_teachers = match[2]
    industry_mentor = match[3]
    foreign_teachers = match[4]
    retirees = match[5]
    affiliated_personnel = match[6]
    full_time_teachers = match[7]
    adm_personnel = match[8]
    supporting_staff = match[9]
    workers = match[10]
    full_time_researchers = match[11]
    subsidiary_personnel = match[12]
    
    print(f"{category:<35} {total:<8} {part_time_teachers:<8} {industry_mentor:<8} {foreign_teachers:<8} {retirees:<8} {affiliated_personnel:<8} {full_time_teachers:<8} {adm_personnel:<8} {supporting_staff:<8} {workers:<8} {full_time_researchers:<8} {subsidiary_personnel:<8}")

ModuleNotFoundError: No module named 'bs4'

In [ ]:

import json
from typing import Optional

from langchain.chains.structured_output import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)


dog_schema = {
    "type": "function",
    "json": {
        "name": "record_dog",
        "description": "Record some identifying information about a dog.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "description": "The dog's name",
                    "type": "string"
                },
                "color": {
                    "description": "The dog's color",
                    "type": "string"
                },
                "fav_food": {
                    "description": "The dog's favorite food",
                    "type": "string"
                }
            },
            "required": ["name", "color"]
        }
    }
}

# structured_llm = create_structured_output_runnable(dog_schema, llm, mode="openai-json",enforce_function_usage=False)
structured_llm = llm.with_structured_output(dog_schema,method="json_mode",include_raw=False)
system = '''Extract information about any dogs mentioned in the user input.'''
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"),]
)
chain = prompt | structured_llm
res = chain.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
print(res)
response = json.dumps(res)
print(response)

{}
{}


In [142]:
from typing import Optional

from langchain.chains.structured_output import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field



llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)


class Dog(BaseModel):
    '''Identifying information about a dog.'''

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")

structured_llm = create_structured_output_runnable(Dog.schema, llm, mode="openai-json",enforce_function_usage=False)
system = '''Extract information about any dogs mentioned in the user input.'''
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"),]
)
chain = prompt | structured_llm
res = chain.invoke({"input": "Harry was a chubby brown beagle who loved chicken"})
print(res)
response = Dog.validate(res)
print(response)

{}


ValidationError: 2 validation errors for Dog
name
  field required (type=value_error.missing)
color
  field required (type=value_error.missing)

In [25]:

import re
import json

from typing import List, Optional

from langchain.chains.structured_output import create_structured_output_runnable
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
#from pydantic import BaseModel, Field, ConfigDict
from langchain_core.messages import AIMessage



llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key= "openai_api_key",
    model="llama3",
)


class Dog(BaseModel):
    '''Identifying information about a dog.'''

    name: str = Field(..., description="The dog's name")
    color: str = Field(..., description="The dog's color")
    fav_food: Optional[str] = Field(None, description="The dog's favorite food")


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: ```json\n{schema}\n```. "
            "Make sure to wrap the answer in ```json and ``` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=Dog.schema())

# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between ```json and ``` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"```json(.*?)```"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")
query = "Harry was a chubby brown beagle who loved chicken"
print(prompt.format_prompt(query=query).to_string())

chain = prompt | llm | extract_json
chain.invoke({"query": query})


System: Answer the user query. Output your answer as JSON that  matches the given schema: ```json
{'title': 'Dog', 'description': 'Identifying information about a dog.', 'type': 'object', 'properties': {'name': {'title': 'Name', 'description': "The dog's name", 'type': 'string'}, 'color': {'title': 'Color', 'description': "The dog's color", 'type': 'string'}, 'fav_food': {'title': 'Fav Food', 'description': "The dog's favorite food", 'type': 'string'}}, 'required': ['name', 'color']}
```. Make sure to wrap the answer in ```json and ``` tags
Human: Harry was a chubby brown beagle who loved chicken


[{'title': 'Dog',
  'description': 'Identifying information about a dog.',
  'type': 'object',
  'properties': {'name': {'title': 'Name',
    'description': "The dog's name",
    'type': 'string'},
   'color': {'title': 'Color',
    'description': "The dog's color",
    'type': 'string'},
   'fav_food': {'title': 'Fav Food',
    'description': "The dog's favorite food",
    'type': 'string'}},
  'required': ['name', 'color']}]

In [42]:
import json

def extract_structured_output(content):
    # Split the content based on schema and answer sections
    sections = content.split('```json\n')

    # Extract JSON content from the answer section
    json_content = sections[2].split('\n```')[0]

    # Parse the JSON
    data = json.loads(json_content)

    # Extract the relevant fields
    action_name = data['action_name']
    is_action_valid = data['is_action_valid']
    new_plan = [item['string'] for item in data['new_plan']]

    # Return the structured output
    structured_output = {
        'action_name': action_name,
        'is_action_valid': is_action_valid,
        'new_plan': new_plan
    }

    return structured_output

# 测试函数
content='''
content='Based on the given schema, I will output my answer as JSON that must match the given schema:\n\n```json\n{\n  "description": "Plans for the next action to be executed by the agent.",\n  "properties": {\n    "action_name": {\n      "description": "Selects the action name of the next action to be executed from the list of available action names.",\n      "title": "Action Name",\n      "type": "string"\n    },\n    "is_action_valid": {\n      "description": "Determines whether the next action is valid or not.",\n      "title": "Is Action Valid",\n      "type": "boolean"\n    },\n    "is_action_valid_reason": {\n      "anyOf": [\n        {"type": "string"},\n        {"type": "null"}\n      ],\n      "default": None,\n      "description": "Then explains the rationale of whether it is valid or not valid action.",\n      "title": "Is Action Valid Reason"\n    },\n    "new_plan": {\n      "description": "The new plan to execute to achieve the goals.",\n      "items": {"type": "string"},\n      "title": "New Plan",\n      "type": "array"\n    }\n  },\n  "required": ["action_name", "is_action_valid", "new_plan"],\n  "title": "PlanNextAction",\n  "type": "object"\n}\n```\n\nHere is my answer:\n\n```json\n{\n  "action_name": "AgentLikesNewsAction",\n  "is_action_valid": true,\n  "is_action_valid_reason": null,\n  "new_plan": [\n    {"string": "Like news article on topic X"},\n    {"string": "Comment on the same news article"}\n  ]\n}\n```\n\nIn this plan, I have selected the action name as `AgentLikesNewsAction`, which is a valid action. The reason for selecting this action is that it aligns with my goals of capturing different views and reactions of students with different academic qualifications and majors to news events. By liking the news article, I am expressing my interest in the topic and showing that I care about what\'s happening in the world.\n\nThe new plan consists of two actions: liking the news article on topic X and commenting on the same news article. These actions will help me achieve my goals by engaging with the news content and sharing my thoughts and opinions with others.' response_metadata={'token_usage': {'completion_tokens': 489, 'prompt_tokens': 0, 'total_tokens': 489}, 'model_name': 'llama3', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None} id='run-119e884c-15b1-4ab1-b4c9-bde96fa85b60-0'
'''

output = extract_structured_output(content)
print(output)


{'action_name': 'AgentLikesNewsAction', 'is_action_valid': True, 'new_plan': ['Like news article on topic X', 'Comment on the same news article']}


In [1]:
import os
import openai
import requests
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])
llm = ChatOpenAI(
    base_url="https://gateway.ai.cloudflare.com/v1/7e0341d53ed432d11bf6df9c07f19ff6/openai/openai",
    model="gpt-3.5-turbo",
    api_key="",
    temperature=0
    )
chain = prompt | llm 
chain.invoke({"input": "how can langsmith help with testing?"})

KeyboardInterrupt: 

{'emotions': {'happiness': 0.2, 'sadness': 0.4}, 'attitudes': {'optimism': 0.3, 'pessimism': 0.5, 'neutrality': 0.2}}


In [1]:
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

print(documents)

["Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels", 'Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands', 'Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall', 'Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight', 'Llamas are vegetarians and have very efficient digestive systems', 'Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old']


In [53]:
import re

def parse_document(doc):
    lines = doc.strip().split('\n')
    result = []

    def add_to_result(level, content):
        current = result
        for _ in range(level - 1):
            current = current[-1][1]
        current.append((content, []))

    current_level = 1
    for line in lines:
        stripped_line = line.strip()
        if not stripped_line or stripped_line.startswith("<!--"):
            continue
        if stripped_line.startswith("#"):
            level = stripped_line.count("#")
            content = stripped_line.lstrip("#").strip()
            add_to_result(level, content)
            current_level = level
        elif re.match(r'^\d+\.\s*\*\*', stripped_line):
            content = re.sub(r'^\d+\.\s*\*\*|\*\*$', '', stripped_line).strip()
            add_to_result(current_level + 1, content)
        elif re.match(r'^-\s', stripped_line):
            content = stripped_line.lstrip("-").strip()
            add_to_result(current_level + 2, content)

    return result

def print_result(result, level=1):
    for item, children in result:
        print("  " * (level - 1) + f"Level {level}: {item}")
        print_result(children, level + 1)

def extract_level(parsed_result, target_level, current_level=1):
    extracted_data = []

    def traverse(result, level):
        nonlocal extracted_data
        for item, children in result:
            if level == target_level:
                extracted_data.append(item)
            else:
                traverse(children, level + 1)

    traverse(parsed_result, current_level)
    return extracted_data



# 示例文档内容
doc = """
# Education Statistics for 2022

## Total Students: 58,030,769

1. **Postgraduates: 3,653,613**
   - Doctor's Degree: 556,065
   - Master's Degree: 3,097,548
2. **Undergraduates: 19,656,436**
3. **Vocational Undergraduate: 16,937,739**
   - Bachelor's Degree: 228,740
   - Associate Degree: 16,708,999
4. **Adult Undergraduate: 9,336,481**
   - Normal Courses: 5,277,598
   - Specialized Courses: 4,058,883
5. **Web-based Undergraduates: 8,446,500**
   - Normal Courses: 3,419,642
   - Short-cycle: 5,026,858

## Total Teachers: 3,450,000

1. **Educational Personnel: 2,870,866**
   - Full-time Teachers: 2,005,188
   - Administrative Personnel: 405,420
   - Supporting Staff: 245,438
   - Workers: 122,982
   - Full-time Researchers: 50,600
   - Other Agency Personnel: 41,238
2. **Others: 579,134**

<!-- # Total Enrollment in Chinese Higher Education

- Total number of students: 58,000,000
- Total number of teachers: 3,500,000
- Student-teacher ratio in Chinese higher education: 17/1 -->

"""
print(type(doc))

parsed_result = parse_document(doc)
print_result(parsed_result)

# 示例调用
level_3_data = extract_level(parsed_result, 4)
print(f"Level 3 Data: {level_3_data}")

<class 'str'>
Level 1: Education Statistics for 2022
  Level 2: Total Students: 58,030,769
    Level 3: Postgraduates: 3,653,613
      Level 4: Doctor's Degree: 556,065
      Level 4: Master's Degree: 3,097,548
    Level 3: Undergraduates: 19,656,436
    Level 3: Vocational Undergraduate: 16,937,739
      Level 4: Bachelor's Degree: 228,740
      Level 4: Associate Degree: 16,708,999
    Level 3: Adult Undergraduate: 9,336,481
      Level 4: Normal Courses: 5,277,598
      Level 4: Specialized Courses: 4,058,883
    Level 3: Web-based Undergraduates: 8,446,500
      Level 4: Normal Courses: 3,419,642
      Level 4: Short-cycle: 5,026,858
  Level 2: Total Teachers: 3,450,000
    Level 3: Educational Personnel: 2,870,866
      Level 4: Full-time Teachers: 2,005,188
      Level 4: Administrative Personnel: 405,420
      Level 4: Supporting Staff: 245,438
      Level 4: Workers: 122,982
      Level 4: Full-time Researchers: 50,600
      Level 4: Other Agency Personnel: 41,238
    Level 3: 

Level 1: Education Statistics for 2022
  Level 2: Total Students: 58,030,769
    Level 3: Postgraduates: 3,653,613
      Level 4: Doctor's Degree: 556,065
      Level 4: Master's Degree: 3,097,548
      Level 4: Bachelor's Degree: 228,740
      Level 4: Associate Degree: 16,708,999
      Level 4: Normal Courses: 5,277,598
      Level 4: Specialized Courses: 4,058,883
      Level 4: Normal Courses: 3,419,642
      Level 4: Short-cycle: 5,026,858
      Level 4: Full-time Teachers: 2,005,188
      Level 4: Administrative Personnel: 405,420
      Level 4: Supporting Staff: 245,438
      Level 4: Workers: 122,982
      Level 4: Full-time Researchers: 50,600
      Level 4: Other Agency Personnel: 41,238
    Level 3: Undergraduates: 19,656,436
    Level 3: Vocational Undergraduate: 16,937,739
      Level 4: Doctor's Degree: 556,065
      Level 4: Master's Degree: 3,097,548
      Level 4: Bachelor's Degree: 228,740
      Level 4: Associate Degree: 16,708,999
      Level 4: Normal Courses: 5,27

In [7]:
import re

def parse_output(output_text):
    # Define keywords
    emotions_keywords = ["happiness", "sadness", "anger"]
    attitudes_keywords = ["optimism", "pessimism", "neutrality"]
    
    emotions = {}
    attitudes = {}

    # Function to extract key-value pairs from a given section
    def extract_values(section):
        values = {}
        pairs = section.split(',')
        for pair in pairs:
            key, value = pair.split(':')
            key = key.strip().strip("'\"")
            value = value.strip()
            values[key] = float(value)
        return values

    # Find and extract emotions
    emotions_section = re.search(r"emotions:\s*\{([^}]*)\}", output_text, re.IGNORECASE)
    if emotions_section:
        emotions = extract_values(emotions_section.group(1))

    # Find and extract attitudes
    attitudes_section = re.search(r"attitudes:\s*\{([^}]*)\}", output_text, re.IGNORECASE)
    if attitudes_section:
        attitudes = extract_values(attitudes_section.group(1))

    return {"emotions": emotions, "attitudes": attitudes}

# Example usage:
output_text = """
You are Teacher Group, Representing 3.5 million Chinese teachers, reflecting their emotions, attitudes and possible actions in response to the news.

emotions: {'happiness': 0.2, 'sadness': 0.4, 'anger': 0.4}
attitudes: {'optimism': 0.1, 'pessimism': 0.5, 'neutrality': 0.4}
"""

parsed_output = parse_output(output_text)
print(parsed_output)


{'emotions': {'happiness': 0.2, 'sadness': 0.4, 'anger': 0.4}, 'attitudes': {'optimism': 0.1, 'pessimism': 0.5, 'neutrality': 0.4}}
